In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
output_path = './output.csv'

In [3]:
user = pd.read_csv('input.csv')

user = user.set_index(pd.DatetimeIndex(user['Time']))

user.drop(columns='Time', inplace=True)

print(user.head(5))
print(user.shape)

                         App_Name  Inactivity Duration(s)  Keystroke_Count  \
Time                                                                         
2021-09-10 13:30:00  explorer.exe             5341.912109              820   
2021-09-10 14:00:00  explorer.exe             3371.500244               87   
2021-09-10 14:30:00  explorer.exe             3674.169434              272   
2021-09-10 15:00:00  explorer.exe             2728.472412               87   
2021-09-10 15:30:00  explorer.exe             2246.260498               87   

                     Speed(ms)  PAM_Val  example_id  
Time                                                 
2021-09-10 13:30:00   7.348455        2           0  
2021-09-10 14:00:00   6.596720        2           0  
2021-09-10 14:30:00   4.375814        2           0  
2021-09-10 15:00:00   3.832777        2           0  
2021-09-10 15:30:00   3.917490        2           0  
(800000, 6)


In [4]:
print(user.PAM_Val.value_counts(normalize=True).sort_index())

1    0.054852
2    0.943225
3    0.001720
4    0.000202
Name: PAM_Val, dtype: float64


In [5]:
user_X = user.drop(columns=['PAM_Val'])
user_y = user.filter(['PAM_Val', 'example_id'])

print(user_X.head(5))
print(user_y.head(5))

user_X = user_X.groupby(by='example_id')
user_y = user_y.groupby(by='example_id')

                         App_Name  Inactivity Duration(s)  Keystroke_Count  \
Time                                                                         
2021-09-10 13:30:00  explorer.exe             5341.912109              820   
2021-09-10 14:00:00  explorer.exe             3371.500244               87   
2021-09-10 14:30:00  explorer.exe             3674.169434              272   
2021-09-10 15:00:00  explorer.exe             2728.472412               87   
2021-09-10 15:30:00  explorer.exe             2246.260498               87   

                     Speed(ms)  example_id  
Time                                        
2021-09-10 13:30:00   7.348455           0  
2021-09-10 14:00:00   6.596720           0  
2021-09-10 14:30:00   4.375814           0  
2021-09-10 15:00:00   3.832777           0  
2021-09-10 15:30:00   3.917490           0  
                     PAM_Val  example_id
Time                                    
2021-09-10 13:30:00        2           0
2021-09-10 14:0

In [6]:
user = []
for one, two in zip(user_X, user_y):
    traj_temp = pd.concat([one[1].drop(columns=['example_id']), two[1]['PAM_Val'].shift(1)], axis=1).fillna(0)
    traj_temp = traj_temp.rename(columns={'PAM_Val': 'Prev_PAM_Val'})
    traj_temp = pd.concat([traj_temp, two[1].drop(columns=['example_id'])], axis=1).fillna(0)
    user.append(traj_temp)

print(np.array(user).shape)

print(user[0])

(100000, 8, 6)
                         App_Name  Inactivity Duration(s)  Keystroke_Count  \
Time                                                                         
2021-09-10 13:30:00  explorer.exe             5341.912109              820   
2021-09-10 14:00:00  explorer.exe             3371.500244               87   
2021-09-10 14:30:00  explorer.exe             3674.169434              272   
2021-09-10 15:00:00  explorer.exe             2728.472412               87   
2021-09-10 15:30:00  explorer.exe             2246.260498               87   
2021-09-10 16:00:00  explorer.exe             2867.938477               87   
2021-09-10 16:30:00  explorer.exe             3240.399658              434   
2021-09-10 17:00:00  explorer.exe             3763.725830              151   

                     Speed(ms)  Prev_PAM_Val  PAM_Val  
Time                                                   
2021-09-10 13:30:00   7.348455           0.0        2  
2021-09-10 14:00:00   6.596720      

In [7]:
counts = [[], []]
for index, traj in enumerate(user):
    #print(index)    
    minority_count = 0
    val_counts = traj.PAM_Val.value_counts().sort_index()
    if 3 in val_counts.index:
        minority_count += val_counts.loc[3]
    if 4 in val_counts.index:
        minority_count += val_counts.loc[4]
    if minority_count >= 4:
        counts[1].append(traj)
    else:
        counts[0].append(traj)

for i in counts:
    print(len(i))


print(pd.concat(counts[1]).PAM_Val.value_counts().sort_index())

99998
2
1    7
2    1
3    8
Name: PAM_Val, dtype: int64


In [8]:
below_min_count_N = min([400, len(counts[0])])
above_min_count_N = min([200, len(counts[1])])

below_min_list = random.sample(counts[0], below_min_count_N)
above_min_list = random.sample(counts[1], above_min_count_N)

full_list = below_min_list + above_min_list

print(len(above_min_list))
print(pd.concat(above_min_list).PAM_Val.value_counts().sort_index())

2
1    7
2    1
3    8
Name: PAM_Val, dtype: int64


In [9]:
seq_len = len(full_list[0])

random.shuffle(full_list)
user_final = pd.concat(full_list)
user_final = user_final.assign(example_id = [index // seq_len for index in range(len(user_final))])

print(user_final.PAM_Val.value_counts(normalize=True).sort_index())

1    0.051617
2    0.945274
3    0.002799
4    0.000311
Name: PAM_Val, dtype: float64


In [10]:
user_final.to_csv(output_path)